## AuxTel Image elongation due to azimuth oscillation

Craig Lage 17-Mar-22

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
from lsst_efd_client import EfdClient
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.bestEffort import BestEffortIsr

In [ ]:
# Get EFD client and the butler
client = EfdClient('usdf_efd')
butler = butlerUtils.makeDefaultLatissButler(embargo=True)
bestEffort = BestEffortIsr(embargo=True)

# Set up the source catalog task
charConfig = CharacterizeImageConfig()
charConfig.doMeasurePsf = False
charConfig.doApCorr = False
charConfig.doDeblend = False
charConfig.repair.doCosmicRay = True
charConfig.repair.doInterpolate = True   
charConfig.detection.minPixels = 500
charTask = CharacterizeImageTask(config=charConfig)

In [ ]:
def find_cwfs(butler, dayObs):
    cwfs_dict = {}
    records = butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs)
    records = records.order_by('id')
    record_iter = iter(records)
    this_record = next(record_iter)
    while True:
        try:
            if this_record.observation_type != 'cwfs':
                this_record = next(record_iter)
                continue
            else:
                this_dict = {}
                
                key = this_record.id
                this_dict['First_CWFS'] = this_record.id
                this_record = next(record_iter)
                while this_record.observation_type == 'cwfs':
                    this_dict['Last_CWFS'] = this_record.id
                    this_record = next(record_iter)
                if this_record.observation_type == 'acq' and int(this_record.exposure_time) == 5:
                    this_dict['CWFS_acq'] = this_record.id
                    this_record = next(record_iter)
                if this_record.observation_type == 'acq' and int(this_record.exposure_time) == 1:
                    this_dict['First_sweep_acq'] = this_record.id
                    this_record = next(record_iter)
                while this_record.observation_type == 'acq':
                    this_record = next(record_iter)
                sweeps = []
                while this_record.observation_type == 'engtest' and int(this_record.exposure_time) == 20:
                    sweeps.append(this_record.id)
                    this_record = next(record_iter)
                if len(sweeps) == 9 and len(this_dict.keys()) == 4:
                    this_dict['Sweeps'] = sweeps
                    cwfs_dict[key] = this_dict
                else:
                    continue
        except StopIteration:
            return cwfs_dict
    return
                
def get_FWHM(expId, charTask): 
    dataId = dict(exposure=expId, detector=0)
    bestEffort.getExposure(dataId)
    dataType = 'quickLookExp'
    exp = butler.get(dataType, dataId)
    charResult = charTask.run(exp)
    sourceCatalog = charResult.sourceCat
    Ixx = sourceCatalog.getIxx()
    Iyy = sourceCatalog.getIyy()
    FWHM_x = 2.35 * np.sqrt(Ixx) 
    FWHM_y = 2.35 * np.sqrt(Iyy) 
    FWHM = np.sqrt((FWHM_x**2 +FWHM_y**2) / 2.0) / 10.0
    print(FWHM)
    print(FWHM_x)
    FWHM_med = np.nanmedian(FWHM)
    FWHM_std = np.nanstd(FWHM)
    return [FWHM_med, FWHM_std]

In [ ]:
get_FWHM(2024040200268, charTask)

In [ ]:
dayObsList = [20240314, 20240321, 20240328]
cwfs_times = []
sweep_times = []

pdf = PdfPages("/home/c/cslage/u/AuxTel/fwhm/Focus_Comparison_01Apr24.pdf")
fig = plt.figure(figsize=(15,5))
for dayObs in dayObsList:
    cwfs_dict = find_cwfs(butler, dayObs)
    for key in cwfs_dict.keys():
        this_dict = cwfs_dict[key]
        cwfs_start_expId = this_dict['First_CWFS']
        mData = butler.get('raw.metadata', detector=0, exposure=cwfs_start_expId)
        cwfs_start = Time(mData['DATE-BEG'], scale='tai')
        cwfs_focus_expId = this_dict['CWFS_acq']
        mData = butler.get('raw.metadata', detector=0, exposure=cwfs_focus_expId)
        cwfs_focus = mData['FOCUSZ']
        cwfs_end = Time(mData['DATE-END'], scale='tai')
        cwfs_time = cwfs_end.unix_tai - cwfs_start.unix_tai
        cwfs_times.append(cwfs_time)
        sweep_start_expId = this_dict['First_sweep_acq']
        mData = butler.get('raw.metadata', detector=0, exposure=sweep_start_expId)
        sweep_start = Time(mData['DATE-BEG'], scale='tai')
        sweep_end_expId = this_dict['Sweeps'][-1]
        mData = butler.get('raw.metadata', detector=0, exposure=sweep_end_expId)
        sweep_end = Time(mData['DATE-BEG'], scale='tai')
        sweep_time = sweep_end.unix_tai - sweep_start.unix_tai 
        sweep_times.append(sweep_time)
        fwhms = []
        stds = []
        focuses = []
        for sweep in this_dict['Sweeps']:
            mData = butler.get('raw.metadata', detector=0, exposure=sweep)
            focus = mData['FOCUSZ']
            [FWHM_med, FWHM_std] = get_FWHM(sweep, charTask)
            focuses.append(focus)
            fwhms.append(FWHM_med)
            stds.append(FWHM_std)
        plt.suptitle(f"CWFS / Focus Sweep comparison {key}", fontsize = 24)
        fig.add_subplot(1,2,1)
        plt.errorbar(focuses, fwhms, yerr=stds, marker = 'x', capsize=2)
        plt.axvline(cwfs_focus, ls = '--', color='red')
        plt.ylim(0.5, 2.0)
        plt.text(cwfs_focus + 0.005, 1.4, "CWFS best focus", color='red')
        plt.text(cwfs_focus + 0.005, 0.55, f"Best focus FWHM = \n{np.min(fwhms):.2f} arcsec", color='green')
        plt.xlabel("Focus_Z (mm)")
        plt.ylabel("Median PSM FWHM (arcseconds)")
        fig.add_subplot(1,2,2)
        plt.axis('off')
        plt.text(0, .9, f"CWFS time = {cwfs_time:.1f} seconds", fontsize=18)
        plt.text(0, .7, f"Sweep time = {sweep_time:.1f} seconds \n(+ analysis - 30 sec min)", fontsize=18)                
        pdf.savefig(fig)  # saves the current figure into a pdf page
        plt.clf()
pdf.close()



In [ ]:
%matplotlib inline
plt.clf()
fig = plt.figure(figsize=(10,5))
plt.title("Focus Methods Comparison March, 2024", fontsize=24)
plt.hist(sweep_times, bins = 50, color='blue')
plt.hist(cwfs_times, bins = 10, color='red')
plt.text(600, 7, f"Median_CWFS_Time = \n{np.median(cwfs_times):.1f} seconds", color='red', fontsize=18)
plt.text(600, 5, f"Median_Sweep_Time = \n{np.median(sweep_times):.1f} seconds (no analysis)", color='blue', fontsize=18)
plt.xlim(0, 2000)
plt.xlabel("Time (sec)")
plt.savefig("/home/c/cslage/u/AuxTel/fwhm/Focus_Time_Comparison_01Apr24.pdf")